In [ ]:
import sys
import os
module_path = os.path.abspath('/media/jary/DATA/Uni/tesi/codice')

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from networks import NoKafnet, Kafnet
import utils.datasetsUtils.CIFAR as CIFAR
from utils.datasetsUtils.taskManager import IncrementalTaskClassification, NoTask
import configs.configClasses as configClasses
from torchvision.transforms import transforms
import torch
import networks.continual_learning as continual_learning
import networks.continual_learning_beta as continual_learning_beta

from Trainer import Trainer
import matplotlib.pyplot as plt
from collections import defaultdict
from copy import deepcopy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [ ]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
config = configClasses.DefaultConfig()
config.L1_REG = 0
config.EPOCHS = 10
config.SAVE_PATH = './models/mt_tecnique_comparison_new'
config.IS_CONVOLUTIONAL = True
config.USE_CL = False
config.MODEL_NAME = ''
config.NEXT_TASK_EPOCHS = None
config.NEXT_TASK_LR = None
config.OPTIMIZER = 'Adam'
# config.DEVICE = 'cpu'
print(config)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

dataset = CIFAR.Cifar100('../data/cifar100', download=True, transform=transform, 
                        task_manager=IncrementalTaskClassification(5),
                        force_download=False, train_split=0.8, target_transform=None)
dataset.load_dataset()
print(len(dataset.Y))

In [ ]:
net = NoKafnet.synCNN(100)

In [ ]:
results = []
experiments = [('no_cont_learning', None),
#                ('ewc', continual_learning.OnlineEWC),
               ('gem', continual_learning.GEM),
               ('embedding', continual_learning_beta.embedding)]

In [ ]:
for name, tec in experiments:
    print(name)
    
    n = deepcopy(net)
    c = deepcopy(config)
    
    c.MODEL_NAME = name

    if name == 'gem':
        c.CL_PAR['margin'] = 0.5
        c.CL_PAR['sample_size'] = 100
        c.USE_CL = True
        c.IS_INCREMENTAL = True
    elif name == 'ewc':
        c.CL_PAR['penalty_importance'] = 50
        c.USE_CL = True
        c.IS_INCREMENTAL = True
    elif name == 'embedding':
        c.CL_PAR = {'penalty_importance': 10, 'weights_type': 'distance', 'sample_size': 100, 'distance': 'cosine', 'supervised': False}
        c.USE_CL = True
        c.IS_INCREMENTAL = True

    c.CL_TEC = tec
    trainer = Trainer(n, deepcopy(dataset), c, save_modality=2, verbose=False, pretrained_model='./models/completi/nokaf')

    r = trainer.load()
    if not r:
        r = trainer.all_tasks()
#     print(r)
    results.append((name, r))
    
    del trainer
    del n

In [ ]:
print('RESULTS')
for name, r in results:
    print(name, r['metrics'])
    print()

In [ ]:
n_task = len(results[0][1]['tasks'])

tot_epochs = 0

for _, r in results:
    for k, v in r['tasks'].items():
        tot_epochs = max(tot_epochs, len(v['accuracy']))
      
fig, ax = plt.subplots(nrows=n_task, ncols=1, figsize=(22, 24), sharex=True, sharey=True)

for name, r in results:
    for i, task in enumerate(r['tasks'].keys()):

        com = r['tasks'][task]
        #no_ewt = metrics_no_ewt['tasks'][task]

        x = range(tot_epochs-len(com['accuracy']), tot_epochs)

        #ax = fig.add_subplot(n_task, 1, i+1, sharex=ax) 

        ax[i].plot(x, com['accuracy'], label=name)
        #ax.plot(x, no_ewt['accuracy'], label='online ewt')

        ax[i].set_xticks(range(0, tot_epochs, 5),minor=False)

        ax[i].set_title("Task {}".format(task))
        ax[i].legend(loc="lower left")
        ax[i].grid(True, axis='both')

        
fig.subplots_adjust(hspace=0.1, wspace=0.01)


In [ ]:
for name, tec in experiments:

    current_w = {n:p.cpu() for n, p in net.named_parameters() if p.requires_grad}

    x = np.arange(4)
    y = np.arange(len(current_w))
    
    xx, yy = np.meshgrid(x, y)
    z = np.zeros(shape=(len(x), len(y)))
    
    fig = plt.figure(figsize=(22, 22))
    ax = fig.add_subplot(111, projection='3d')
    
    x_labels = []
    
    for i in x:

        if i == 0:
            x_labels.append('0')
            
        else:
            x_labels.append('{}->{}'.format(i-1, i))
            
        n = deepcopy(net)
        c = deepcopy(config)

        c.MODEL_NAME = name

        if 'name' == 'gem':
            c.EWC_IMPORTANCE = 0.5

        c.EWC_TYPE = tec

        trainer = Trainer(n, deepcopy(dataset), c, save_modality=2)

        r = trainer.load(i)
        
        diff = {n: torch.dist(p.cpu(), current_w[n]) for n, p in trainer.model.named_parameters() if p.requires_grad }
        current_w = {n: p.cpu() for n, p in trainer.model.named_parameters() if p.requires_grad }
        
        for j, (_, d) in enumerate(diff.items()):
            z[i, j] = d
    

    z_n = z.flatten()

    ax.view_init(45, 45)
    
    ax.bar3d(xx.flatten(),
          yy.flatten(),
          np.zeros(len(z_n)),
          0.2, 0.5, z_n )
    
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels)
    
    ax.set_yticks(y)
    ax.set_yticklabels(list(diff.keys()), fontdict={'fontsize':12})
    
    ax.text(3.5, 0, 2, name.upper(), color='black',  fontsize='xx-large', 
        bbox=dict(facecolor='white', edgecolor='black'))
    
    plt.show()

In [ ]:
tot_epochs = 0

fig, ax = plt.subplots(nrows=len(experiments)-1, ncols=5, figsize=(35, 40), 
                       sharex=True, sharey=True,  subplot_kw={'projection':'3d'})

    
images_c = {}
images = []
labels = []

for k in range(5):
    dataset.task = k
        
    if k == 0:
        for im in range(len(dataset)):

            img, lbl = dataset[(im, dataset.task_mask(k))]
            lbl = lbl.numpy()[0]

            if lbl not in images_c:
                images_c[lbl] = 1
                images.append(img.numpy())
                labels.append(lbl)
            else:
                if images_c[lbl] < 50:
                    images.append(img.numpy())
                    images_c[lbl] = images_c[lbl] + 1
                    labels.append(lbl)
                
        colors = [int(i % 21) for i in labels]

    for i, (name, tec) in enumerate(experiments):
        
        if name == 'no_cont_learning':
            continue 
            
        tsne_model_en_2d = PCA(n_components=3, random_state=19)

        n = deepcopy(net)
        c = deepcopy(config)

        c.MODEL_NAME = name

        c.EWC_TYPE = tec

        trainer = Trainer(n, deepcopy(dataset), c, save_modality=2)
        
        r = trainer.load()
        trainer.model.task = 0

        a = trainer.model.embedding(torch.from_numpy(np.asarray(images)).to(config.DEVICE)).detach().cpu().numpy()
        b = tsne_model_en_2d.fit_transform(a)

        ax[i-1][k].scatter(b[:, 0], b[:, 1], b[:, 2], c=colors)
        ax[i-1][k].set_title("{} embeddings on task: {}".format(name, k))

        del trainer
        del n
    plt.show()